In [1]:
%pip install faiss-cpu sentence-transformers langchain google-generativeai python-dotenv langchain-community flask flask-cors langchain_google_genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings

hf_embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

load_dotenv()

product_vector = FAISS.load_local(
    folder_path="Product_Vector",
    embeddings=hf_embedding,
    allow_dangerous_deserialization=True
)



d:\Projects\GE\server\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\yaswa\AppData\Local\Temp\ipykernel_8576\1982620398.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')


In [3]:
import json
d = [
  'ID:12  Name:pesara pappu | Moong Dal  URL:pesara_pappu.jpg',
  'ID:12  Name:pesara pappu | Moong Dal  URL:pesara_pappu.jpg',
  'ID:18  Name:senaga pappu | Channa Dal  URL:senaga_pappu.jpg'
]

def json_converter(ls):
    
    temp = []
    
    for line in ls:
        item = {}
        for parts in line.split("  "):
            key,val = parts.split(":")
            item[key.strip()] = val.strip()
        temp.append(item)
    return json.dumps(temp,indent=2)

           

In [4]:
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=0)

memory = VectorStoreRetrieverMemory(retriever=product_vector.as_retriever())

coversation = ConversationChain(
  llm=llm,
  memory=memory
)

C:\Users\yaswa\AppData\Local\Temp\ipykernel_8576\2837595436.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=product_vector.as_retriever())
C:\Users\yaswa\AppData\Local\Temp\ipykernel_8576\2837595436.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  coversation = ConversationChain(


In [5]:

res = coversation.predict(input='give me in object formate only item:'+'poppu')

print(res)

Okay, I can provide you with the items that contain "poppu" in their name, in object format. Based on the data I have, here's the information:

```json
[
  {
    "id": 12,
    "name": "pesara pappu | Moong Dal",
    "url": "pesara_pappu.jpg",
    "itemform": "raw"
  },
  {
    "id": 11,
    "name": "Vedaka Popular Toor Dal | kandipappu",
    "url": "kandipappu.jpg",
    "itemform": "raw"
  },
    {
    "id": 18,
    "name": "senaga pappu | Channa Dal",
    "url": "senaga_pappu.jpg",
    "itemform": "raw"
  }
]
```

I've included all the entries where "poppu" or "pappu" appears in the name. Let me know if you need anything else!


In [6]:
from langchain.agents import AgentType,initialize_agent,AgentExecutor
from langchain.tools import Tool
from langchain.memory import ConversationSummaryMemory

def json_converter(ls):
    
    temp = []
    
    for line in ls:
        item = {}
        for parts in line.lower().split("  "):
            key,val = parts.split(":")
            item[key.strip()] = val.strip()
        temp.append(item)
        for i in temp:
          i['quantity']=1
          i['weight'] = 100
    return temp


def sim_wrapper(input_str):
    print(input_str) 
    try:
        if 'k=' in input_str:
            res1,res2 = input_str.split(',')
            return sim(res1.strip(),(int(res2.strip().replace('k=',''))))
        elif not ',' in input_str :
          return sim(input_str)
        else:
          raise ZeroDivisionError
    except:
        return 'input must be like `itemname,k=3` .'
    

def sim(product,k=1):
    print(product,k)
    docs2 = product_vector.similarity_search(product, k=k)
    combined_context = "\n".join([d.page_content for d in docs2])
    
    print(combined_context)
    
    return json_converter(combined_context.lower().split('\n'))
  
search_tool = Tool(
   name='SearchProducts',
   func=sim_wrapper,
    description=(
        "Use this tool to search for products. It takes two arguments: "
        "1) product name (string), and "
        "2) k (number of results to return). "
        "If user asks for product details or examples (like 'what Pappu items do you have?'), set k value based on preference. "
        "If user asks to add products to cart, return JSON response of those products. "
        "Default k is 1."
    )
)
    
import wikipedia
def wiki_summary(topic:str) -> str:
  try:
    return wikipedia.summary(topic,sentences=2)
  except Exception as e:
    return e
    
wiki_tool = Tool(
  name='WikiSummary',
  func=wiki_summary,
  description="Use this to get short summary on any product from wikipedia. Input should be the item name."
)

def chat_bot(query):
  response = llm.invoke(query)
  return str(response)

chat_tool = Tool(
  name="GeminiChat",
  func=chat_bot,
  description="Use this tool for general coversation ,answering question,checking products types like oil, liquid, solid or not ,etc.."
)

In [7]:
val = sim('pappu',5)
  
print(val)


pappu 5
id:12  name:pesara pappu | Moong Dal  url:pesara_pappu.jpg  itemform:raw
id:18  name:senaga pappu | Channa Dal  url:senaga_pappu.jpg  itemform:raw
id:11  name:Vedaka Popular Toor Dal | kandipappu   url:kandipappu.jpg  itemform:raw
id:13  name:No Preservatives Whole Moong | pesalu  url:pesalu.jpg  itemform:raw
id:8  name:Bague Premium Whole Methi Seeds Dana Fenugreek seed | menthulu  url:menthulu.jpg  itemform:raw
[{'id': '12', 'name': 'pesara pappu | moong dal', 'url': 'pesara_pappu.jpg', 'itemform': 'raw', 'quantity': 1, 'weight': 100}, {'id': '18', 'name': 'senaga pappu | channa dal', 'url': 'senaga_pappu.jpg', 'itemform': 'raw', 'quantity': 1, 'weight': 100}, {'id': '11', 'name': 'vedaka popular toor dal | kandipappu', 'url': 'kandipappu.jpg', 'itemform': 'raw', 'quantity': 1, 'weight': 100}, {'id': '13', 'name': 'no preservatives whole moong | pesalu', 'url': 'pesalu.jpg', 'itemform': 'raw', 'quantity': 1, 'weight': 100}, {'id': '8', 'name': 'bague premium whole methi seeds

In [8]:
agent = initialize_agent(
   tools=[search_tool,wiki_tool,chat_tool],
   llm=llm,
   agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   system_message=(
        "You must always use the SearchProducts tool with input example: `pappu,k=2` ."
        "Do not use Python dictionaries or any other format. Only JSON strings."
        "You must always use the SearchProducts tool when the user asks about product availability or product details."
        "If user asks to add products to the cart, return the output in JSON format. "
        "If user only asks for information, give a text response summarizing the search results."  
    )
)

C:\Users\yaswa\AppData\Local\Temp\ipykernel_8576\2394252333.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [9]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)

agent_executer = AgentExecutor.from_agent_and_tools(
  tools=[wiki_tool,search_tool,chat_tool],
  memory=memory,
  agent=agent.agent,
  verbose=True
)

C:\Users\yaswa\AppData\Local\Temp\ipykernel_8576\2833518875.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)


In [10]:
res = agent_executer.run('find pesara pappu add it to cart')
print(res)

C:\Users\yaswa\AppData\Local\Temp\ipykernel_8576\2194290350.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = agent_executer.run('find pesara pappu add it to cart')




> Entering new AgentExecutor chain...
The user wants to add "pesara pappu" to the cart. I need to find the product and return a JSON response.
Action: SearchProducts
Action Input: pesara pappupesara pappu
pesara pappu 1
id:12  name:pesara pappu | Moong Dal  url:pesara_pappu.jpg  itemform:raw

Observation: [{'id': '12', 'name': 'pesara pappu | moong dal', 'url': 'pesara_pappu.jpg', 'itemform': 'raw', 'quantity': 1, 'weight': 100}]
Thought:I have found the product "pesara pappu | moong dal". Now I need to return the JSON response.
Final Answer: ```json
[{'id': '12', 'name': 'pesara pappu | moong dal', 'url': 'pesara_pappu.jpg', 'itemform': 'raw', 'quantity': 1, 'weight': 100}]
```

> Finished chain.
```json
[{'id': '12', 'name': 'pesara pappu | moong dal', 'url': 'pesara_pappu.jpg', 'itemform': 'raw', 'quantity': 1, 'weight': 100}]
```


In [11]:
import re
import json

pattern = r'```json\s*(.*?)\s*```'
match = re.search(pattern, res, re.DOTALL)

if match:
    json_text = match.group(1).strip()
    parsed_json = json.loads(json_text)
    print(parsed_json)
else:
    print("No JSON block found.")


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [26]:
print(sim("",10))

 10
[
  {
    "ID": "14",
    "Name": "Vedaka Whole Jeera | gelakarra",
    "URL": "gelakarra.jpg"
  },
  {
    "ID": "16",
    "Name": "Dry Fruit Hub Copra Dry Coconut",
    "URL": "Dry_Coconut.jpg"
  },
  {
    "ID": "11",
    "Name": "Vedaka Popular Toor Dal | kandipappu",
    "URL": "kandipappu.jpg"
  },
  {
    "ID": "4",
    "Name": "sugar",
    "URL": "sugar.jpg"
  },
  {
    "ID": "13",
    "Name": "No Preservatives Whole Moong | pesalu",
    "URL": "pesalu.jpg"
  },
  {
    "ID": "9",
    "Name": "elakulu | Emperor Akbar GI Tagged Alleppey Green Cardamom | Elaichi",
    "URL": "Elaichi.jpg"
  },
  {
    "ID": "18",
    "Name": "senaga pappu | Channa Dal",
    "URL": "senaga_pappu.jpg"
  },
  {
    "ID": "15",
    "Name": "Vedaka Poppy Seeds | gasagasalu",
    "URL": "gasagasalu.jpg"
  },
  {
    "ID": "8",
    "Name": "Bague Premium Whole Methi Seeds Dana Fenugreek seed | menthulu",
    "URL": "menthulu.jpg"
  },
  {
    "ID": "1",
    "Name": "Aashirvaad Sharbati Atta",
    "

In [55]:
from langchain.agents import AgentType,initialize_agent,AgentExecutor
from langchain.tools import Tool
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
import wikipedia
import json
import re

load_dotenv()

class Tools_Part():
  
  def __init__(self):
    self.llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=0)
    self.search_tool = Tool(
        name='SearchProducts',
        func=self.sim_wrapper,
         description=(
             "Use this tool to search for products. It takes two arguments: "
             "1) product name (string), and "
             "2) k (number of results to return). "
             "If user asks for product details or examples (like 'what Pappu items do you have?'), set k value based on preference. "
             "If user asks to add products to cart, return JSON response of those products. "
             "Default k is 1."
             "If query what to see image include url in responce example: url:'dummy.jpg'"
         )
        )
    
    self.wiki_tool = Tool(
           name='WikiSummary',
           func=self.wiki_summary,
           description="Use this to get short summary on any product from wikipedia. Input should be the item name."
         )
    
    self.chat_tool = Tool(
        name="GeminiChat",
        func=self.chat_bot,
        description="Use this tool for general coversation ,answering question,checking products types like oil, liquid,solid or not ,etc.."
      )
    
    self.weight_convert = Tool(
        name="WeightCovert",
        func=self.parse_weight,
        description="Use this tool for to get Weights in grams from liter to grams, kg to grams, eg: input must be like : 1kg or (value)liter or (value)g"
      )
    
    self.hf_embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
    
    self.product_vector = FAISS.load_local(
    folder_path="Product_Vector",
    embeddings=self.hf_embedding,
    allow_dangerous_deserialization=True
)
    
  
  def json_converter(self,ls):
    
    temp = []
    
    for line in ls:
        item = {}
        for parts in line.lower().split("  "):
            key,val = parts.split(":")
            item[key.strip()] = val.strip()
        temp.append(item)
        for i in temp:
          i['quantity']=1
          i['weight'] = 0
    return temp


  def sim_wrapper(self,input_str):
      print(input_str) 
      try:
          if 'k=' in input_str:
              res1,res2 = input_str.split(',')
              return self.sim(res1.strip(),(int(res2.strip().replace('k=',''))))
          elif not ',' in input_str :
            return self.sim(input_str)
          else:
            raise ZeroDivisionError
      except:
          return 'input must be like `itemname,k=3` .'
      
  
  def sim(self,product,k=1):
      print(product,k)
      docs2 = self.product_vector.similarity_search(product, k=k)
      combined_context = "\n".join([d.page_content for d in docs2])
      
      return self.json_converter(combined_context.split('\n'))
    
  def wiki_summary(self,topic:str) -> str:
     try:
       return wikipedia.summary(topic,sentences=2)
     except Exception as e:
       return e
  
  def chat_bot(self,query):
     response = self.llm.invoke(query)
     return str(response)
   
  def pure(self,res):
    pattern = r'```json\s*(.*?)\s*```'
    match = re.search(pattern, res, re.DOTALL)
    
    if match:
        json_text = match.group(1).strip()
        parsed_json = json.loads(json_text)
        return parsed_json
    elif "[{'id':" in res:
        return json.loads(res.replace('\'',"\""))
    else:
      return "No Object Found"
    
  def parse_weight(self,weight_text):
    """
    Parse weight text like '1kg' or '1liter' and return numeric weight:
    - 1kg ➜ 1000
    - 1liter ➜ 910
    - 500g ➜ 500
    - Plain numbers remain as is.
    """
    weight_text = weight_text.lower().strip()
    if weight_text.endswith('kg'):
        num = float(weight_text.replace('kg', '').strip())
        return int(num * 1000)  # 1kg=1000g
    elif weight_text.endswith('g'):
        num = float(weight_text.replace('g', '').strip())
        return int(num)
    elif weight_text.endswith('liter'):
        num = float(weight_text.replace('liter', '').strip())
        return int(num * 910)  # 1 liter = 910g (as per your case)
    else:
        try:
            return int(weight_text)
        except:
            return 100  # fallback
          
      
class Agent_Part(Tools_Part):
  
  def __init__(self):
   super().__init__()
    
   self.memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)
   
   self.agent = initialize_agent(
    tools=[self.search_tool, self.wiki_tool,self.chat_tool,self.weight_convert],
    llm=self.llm,
    memory=self.memory,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    system_message=(
        "Rules You Must Follow for each Item this must only contain : id is item id ,name is item name,url is image url,itemform type of item form,quantity is number of items ,weight is weight of the item example ```json {'id': 'itemid', 'name': 'itemname', 'url': 'url', 'itemform': 'raw', 'quantity': 1, 'weight': 100} ```"
        "Item weight must be only in grams"
        "You must always use the SearchProducts tool with input example: `pappu,k=2`."
        "Do not use Python dictionaries or any other format. Only JSON strings."
        "You must always use the SearchProducts tool when the user asks about product availability or product details."
        "If the user asks to add products to the cart, return the output in JSON format. example ```json {'id': 'itemid', 'name': 'itemname', 'url': 'url', 'itemform': 'raw', 'quantity': 1, 'weight': 100} ```"
        "If the user only asks for information, give a text response summarizing the search results."
        "You must only follow items in this formate example ```json {'id': 'itemid', 'name': 'itemname', 'url': 'url', 'itemform': 'raw', 'quantity': 1, 'weight': 100} ```"
        "weight must be in grams only"
    )
)


   self.agent_executer = AgentExecutor.from_agent_and_tools(
     memory=self.memory,
     agent=self.agent.agent,
     tools=[self.search_tool,self.wiki_tool,self.chat_tool,self.weight_convert],
     verbose=True 
   )


In [58]:
agent_obj = Agent_Part()

print(agent_obj.agent_executer.run('add 2 1liter vijaya oil'))




> Entering new AgentExecutor chain...
The user wants to add 2 1-liter Vijaya oil to the cart. I need to find the product "Vijaya oil" and return the JSON response for 2 quantities of 1-liter size.

Action: SearchProducts
Action Input: Vijaya oilVijaya oil
Vijaya oil 1

Observation: [{'id': '2', 'name': 'groundnut oil vijaya', 'url': 'groundnut_oil.webp', 'itemform': 'oil', 'quantity': 1, 'weight': 0}]
Thought:The user wants to add 2 1-liter Vijaya oil to the cart. The search result shows "groundnut oil vijaya" with a quantity of 1. I need to update the quantity to 2 and specify the weight as 1 liter. Since the weight is currently 0, I need to convert 1 liter to grams to maintain consistency.

Action: WeightCovert
Action Input: 1liter
Observation: 910
Thought:I now know the weight of 1 liter of oil is 910 grams. I can now construct the JSON response with the correct quantity and weight.

Action: SearchProducts
Action Input: groundnut oil vijayagroundnut oil vijaya
groundnut oil vijaya

In [34]:
print(agent_obj.weight_convert('1.1liter'))

1001
